In [1]:
'''
Movie and Book Chatbot

Keeps track of your movies and books, their ratings, and outputs recommendations based on NLP
'''

import nltk
import re
import pickle
from datetime import datetime
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from collections import defaultdict

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\myaku\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\myaku\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\myaku\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
def check_strings_present(string_list, text):
    for string in string_list:
        if string not in text.lower():
            return False
    return True

In [3]:
def interact_with_chatbot(assistant, user_id):
    print("Welcome to the Personal Book and Movie chatbot!")
    print("Type 'quit' to exit.")

    while True:
        user_input = input(f"{user_id}: ")
        if user_input.lower() == 'quit':
            break

        response = assistant.respond(user_id, user_input)
        print(f"Assistant: {response}")

    assistant.save_user_data()
    print("Goodbye!")

In [4]:
class UserModel:
    def __init__(self, name):
        self.name = name
        self.book_reviews = {}
        self.movie_reviews = {}
        self.name_prompt = False

In [5]:
class PersonalAssistant:
    def __init__(self, user_data_file):
        self.user_data_file = user_data_file
        self.users = self.load_user_data()
    
    def load_user_data(self):
        try:
            with open(self.user_data_file, 'rb') as f:
                return pickle.load(f)
        except (FileNotFoundError, EOFError):
            return {}

    def save_user_data(self):
        with open(self.user_data_file, 'wb') as f:
            pickle.dump(self.users, f)

    def get_user_model(self, user_id):
        if user_id not in self.users:
            self.users[user_id] = UserModel("unknown")
        return self.users[user_id]
    
    def update_user_model(self, user_model, text):
        
#         if "name" in text.lower():
#             tagged_text = pos_tag(word_tokenize(text))
#             for word, pos in tagged_text:
#                 if pos == 'NNP':
#                     user_model.name = word.capitalize()

        ratings = re.findall(r'I rate (book|movie) (\w+) (\d+(\.\d{1,2})?)/10', text, re.IGNORECASE)
        for category, item, rating in ratings:
            if category.lower() == 'book':
                user_model.book_ratings[item.lower()] = float(rating)
            elif category.lower() == 'movie':
                user_model.movie_ratings[item.lower()] = float(rating)

    def respond(self, user_id, text):
        user_model = self.get_user_model(user_id)
        self.update_user_model(user_model, text)
        
        if user_model.name_prompt:
            tagged_text = pos_tag(word_tokenize(text))
            for word, pos in tagged_text:
                if pos == 'NNP':
                    user_model.name = word.capitalize()
            if user_model.name == 'unknown':
                user_model.name = text
            user_model.name_prompt = False
            return f'Alright, nice to meet you {user_model.name}'
        
        if user_model.name == 'unknown':
            user_model.name_prompt = True
            return f"Hello!, I've noticed I don't know your name in my knowledge base. Could you provide it so I know how to address you?"
            
        if check_strings_present(['change', 'name'], text):
            user_model.name_prompt = True
            return f'Alright, let me know what you want to change your name to'
            
        if check_strings_present(['what', 'name'], text):
            return f"Your name is {user_model.name}"      
            
        if "ratings" in text.lower():
            category = re.search(r'ratings for (\w+)', text, re.IGNORECASE)
            if category and category.group(1).lower() in ['books', 'movies']:
                if category.group(1).lower() == 'books':
                    items = user_model.book_ratings
                else:
                    items = user_model.movie_ratings
                response = f"{user_model.name}, your ratings for {category.group(1).lower()} are:\n"
                for item, rating in items.items():
                    response += f"- {item}: {rating}/10\n"
                return response
            return f"I couldn't find any ratings for {category.group(1).lower()} for you, {user_model.name}."
            
        if any(user_input in text for user_input in ['hello', 'hi', 'greetings']):
            return f"Hello, {user_model.name}! How can I help you today?" 

        return f"I'm not sure what you're asking, {user_model.name}. Could you please rephrase?"

In [ ]:
user_id = "Matthew Y"
user_data_file = user_id + '.pkl'
assistant = PersonalAssistant(user_data_file)

interact_with_chatbot(assistant, user_id)

Welcome to the Personal Book and Movie chatbot!
Type 'quit' to exit.
Matthew Y: test
Assistant: Hello!, I've noticed I don't know your name in my knowledge base. Could you provide it so I know how to address you?
Matthew Y: Matthew
Assistant: Alright, nice to meet you Matthew
Matthew Y: change my name to Drew
Assistant: Alright, let me know what you want to change your name to
Matthew Y: Drew
Assistant: Alright, nice to meet you Drew
